In [15]:
import pandas as pd

In [16]:
DATA_PATH = '../data/raw/'

calendar = pd.read_csv(DATA_PATH + "calendar.csv")
sell_prices = pd.read_csv(DATA_PATH + "sell_prices.csv")
usecols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"] + [f"d_{i}" for i in range(1, 1914)]
sales_train_val = pd.read_csv(DATA_PATH + "sales_train_validation.csv", usecols=usecols)

sales_train_val = sales_train_val.query("state_id == 'CA'").copy()
print(calendar.shape, sell_prices.shape, sales_train_val.shape)

(1969, 14) (6841121, 4) (12196, 1919)


In [17]:
sales_long = sales_train_val.melt(
    id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
    var_name = 'd',
    value_name = 'sales'
)

In [19]:
sales_long = sales_long.merge(
    calendar[['date', 'd', 'wm_yr_wk', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']],
    on = 'd', how='left'
)
sales_long['date'] = pd.to_datetime(sales_long['date'])

In [20]:
sales_long = sales_long.merge(
    sell_prices,
    on=["store_id", "item_id", "wm_yr_wk"],
    how="left"
)

In [21]:
del sales_train_val

In [22]:
missing = sales_long.isnull().mean().sort_values(ascending=False)
print(missing[missing > 0])

event_type_2    0.997909
event_name_2    0.997909
event_name_1    0.919498
event_type_1    0.919498
sell_price      0.219157
dtype: float64


In [23]:
sales_long['event_name_1'].fillna('None', inplace=True)
sales_long['event_type_1'].fillna('None', inplace=True)
sales_long['event_name_2'].fillna('None', inplace=True)
sales_long['event_type_2'].fillna('None', inplace=True)
sales_long['sell_price'].fillna(method='ffill', inplace=True)

sales_long = sales_long.dropna(subset=['date'])

C:\Users\srmpc\AppData\Local\Temp\ipykernel_19164\495715243.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sales_long['event_name_1'].fillna('None', inplace=True)
C:\Users\srmpc\AppData\Local\Temp\ipykernel_19164\495715243.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [ ]:
sales_long["date"].min(), sales_long["date"].max()

(Timestamp('2011-01-29 00:00:00'), Timestamp('2016-04-24 00:00:00'))

In [27]:
sales_long.duplicated(subset=['id', 'date']).sum()

np.int64(0)

In [28]:
sales_long[sales_long['sales'] < 0].shape

(0, 18)

In [31]:
sales_long['dayofweek'] = sales_long['date'].dt.dayofweek
sales_long['weekofyear'] = sales_long['date'].dt.isocalendar().week
sales_long['month'] = sales_long['date'].dt.month
sales_long['year'] = sales_long['date'].dt.year

In [34]:
subset_ids = sales_long['id'].unique()[:800]
eda_subset = sales_long[sales_long['id'].isin(subset_ids)].copy()

In [37]:
eda_subset.to_parquet("../data/processed/m5_timeseries.parquet", index=False)

In [38]:
print("Subset shape:", eda_subset.shape)

Subset shape: (1530400, 22)
